In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!unzip seg_models.zip
!unzip utils.zip

Archive:  seg_models.zip
replace seg_models/__init__.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  utils.zip
replace utils/__init__.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
!pip install torchio
!pip install torchvision
!pip install pretrainedmodels
!pip install yacs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 172 kB 5.1 MB/s 
     |████████████████████████████████| 52.8 MB 1.2 MB/s 
     |████████████████████████████████| 237 kB 67.9 MB/s 
     |████████████████████████████████| 51 kB 7.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=1c622a07ee1e270804f13b86d9dea016e7193ec0217321e91ce57a1e63b77fa1
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import torch
import torchio as tio
import torch.nn as nn
import torch.optim as optim
import seg_models as sm
import utils

In [7]:
device = (torch.device('cuda') if torch.cuda.is_available()
                  else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [9]:
# infect_path = "data/Infection Segmentation Data Transformed/"
infect_path = "/content/gdrive/MyDrive/Usc/EE641/Project/Data/Infection Segmentation Data Transformed"
infect_train_loader, infect_val_loader, infect_test_loader = utils.get_loader(
    target_directory=infect_path, train_bs=64,
    val_bs=64, num_works=4)
    
# # lung_path = "data/Lung Segmentation Data Transformed/"
# lung_path = "/content/gdrive/MyDrive/Usc/EE641/Project/Data/Lung Segmentation Data Transformed"
# lung_train_loader, lung_val_loader, lung_test_loader = utils.get_loader(
#     target_directory=lung_path, train_bs=64,
#     val_bs=64, num_works=4)
# len(infect_train_loader.dataset), len(lung_train_loader.dataset)

In [10]:
infect_Unet_micronet_m0 = sm.Unet(
    encoder_name="micronet_m0",       
    in_channels=1,                  
    classes=2,                      
)
infect_optimizer = optim.Adam(infect_Unet_micronet_m0.parameters(), lr=0.0001)

lung_Unet_micronet_m0 = sm.Unet(
    encoder_name="micronet_m0",       
    in_channels=1,                  
    classes=2,                      
)
lung_optimizer = optim.Adam(infect_Unet_micronet_m0.parameters(), lr=0.0001)

loss_fcn = nn.CrossEntropyLoss()

### load model to GPU if available
if torch.cuda.is_available():
    infect_Unet_micronet_m0.cuda()

In [12]:
utils.training_loop(
    infect_Unet_micronet_m0, infect_optimizer,
    loss_fcn, infect_train_loader,
    infect_val_loader, mask_name='infection mask',
    n_epochs=20, device=device, 
    save_path='trained_models/infect_Unet_micronet_m0.pt')

Epoch 1 Start:
Epoch 1, Batch 0
Epoch 1, Batch 10
Epoch 1, Batch 20
Epoch 1, Batch 30
Epoch 1, Batch 40
Epoch 1, Batch 50
Epoch 1, train loss: 0.228
Duration: 0.524 minutes
Epoch 1, Batch 0
Epoch 1, val loss: 0.220


RuntimeError: ignored

In [13]:
utils.test_loop(
    infect_Unet_micronet_m0, infect_test_loader,
    mask_name='infection mask', device=device)

patient 0
patient 10
patient 20
patient 30
patient 40
patient 50
patient 60
patient 70
patient 80
patient 90
patient 100
patient 110
patient 120
patient 130
patient 140
patient 150
patient 160
patient 170
patient 180
patient 190
patient 200
patient 210
patient 220
patient 230
patient 240
patient 250
patient 260
patient 270
patient 280
patient 290
patient 300
patient 310
patient 320
patient 330
patient 340
patient 350
patient 360
patient 370
patient 380
patient 390
patient 400
patient 410
patient 420
patient 430
patient 440
patient 450
patient 460
patient 470
patient 480
patient 490
patient 500
patient 510
patient 520
patient 530
patient 540
patient 550
patient 560
patient 570
patient 580
patient 590
patient 600
patient 610
patient 620
patient 630
patient 640
patient 650
patient 660
patient 670
patient 680
patient 690
patient 700
patient 710
patient 720
patient 730
patient 740
patient 750
patient 760
patient 770
patient 780
patient 790
patient 800
patient 810
patient 820
patient 830
pat

(0.9287041425704956, 0.08986986428499222, nan, nan, nan, nan)

In [ ]:
# load lung dataloader later
# lung_path = "data/Lung Segmentation Data Transformed/"
lung_path = "/content/gdrive/MyDrive/Usc/EE641/Project/Data/Lung Segmentation Data Transformed"
lung_train_loader, lung_val_loader, lung_test_loader = utils.get_loader(
    target_directory=lung_path, train_bs=64,
    val_bs=64, num_works=4)
len(infect_train_loader.dataset), len(lung_train_loader.dataset)

In [ ]:
utils.training_loop(
    lung_Unet_micronet_m0, lung_optimizer,
    loss_fcn, infect_train_loader,
    infect_val_loader, mask_name='lung mask',
    n_epochs=20, device=device, 
    save_path='trained_models/lung_Unet_micronet_m0.pt')

In [ ]:
utils.test_loop(
    lung_Unet_micronet_m0, lung_test_loader,
    mask_name='lung mask', device=device)